In [2]:
import cv2 

stream = cv2.VideoCapture(0)

In [8]:
import torch
import requests
from torch import hub # Hub contains other models like FasterRCNN
model = torch.hub.load( \
                      'ultralytics/yolov5', \
                      'yolov5s', \
                      pretrained=True)

ModuleNotFoundError: No module named 'requests'